# Super Image Resolution

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install numpy opencv-python lmdb pyyaml pickle5 matplotlib seaborn tqdm

In [ ]:
apt-get update && apt-get install libgl1-mesa-glx ffmpeg

In [ ]:
!git clone https://github.com/xinntao/ESRGAN

# SIR

In [ ]:
!git clone https://github.com/idealo/image-super-resolution SIR/src

In [ ]:
%cd SIR/src
!pip install -e ".[gpu]"
%cd ../..

In [1]:
################# Required Configurations ############################

INPUT_FILEPATH = "example.mp4" #param{type:"string"}

OUTPUT_FILEPATH = "example4K.mp4" #param{type:"string"}

################# Optional configurations ############################

INPUTFRAMES_DIRPATH = 'SIR/frames' #@param{type:"string"}

OUTPUTFRAMES_DIRPATH = 'SIR/framesHD' #@param{type:"string"}

In [2]:
# Importing all necessary libraries 
import cv2 
import os 
import numpy as np
import glob
from tqdm import tqdm
import os

In [3]:
def runnerISR(path_to_images,path_to_output):
    
    from ISR.models import RDN
    import numpy as np
    from PIL import Image

    files = []

    for r, _, f in os.walk(path_to_images):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","")])
    print("")
    files.sort()
    files.sort(key=lambda x: int(x[1]))
    #-----------------------------------------------
    model = RDN(weights='noise-cancel')
    
    for file in tqdm(range(len(files))):

        path = files[file][0]
        name = files[file][1]

        img = Image.open(path)
        sr_img = model.predict(np.array(img))
        image2x = Image.fromarray(sr_img)
        image2x.save(path_to_output+"/"+name+'.jpg','JPEG')

In [4]:
rm -R '{INPUTFRAMES_DIRPATH}' && rm -R '{OUTPUTFRAMES_DIRPATH}' 

In [5]:
mkdir '{INPUTFRAMES_DIRPATH}'&& mkdir '{OUTPUTFRAMES_DIRPATH}'

In [6]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
filename = os.path.basename(INPUT_FILEPATH)
!ffmpeg -r 23.98 -i '{filename}' '{INPUTFRAMES_DIRPATH}/%05d.png'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
runnerISR(INPUTFRAMES_DIRPATH,OUTPUTFRAMES_DIRPATH)

 17%|█▋        | 236/1364 [05:53<27:56,  1.49s/it]

In [ ]:
%cd {OUTPUTFRAMES_DIRPATH}
! rm 'temp_outputaudio.aac'
! ffmpeg -i '../../{INPUT_FILEPATH}' -acodec copy 'temp_outputaudio.aac'
! ffmpeg -y -r "23.98" -f image2 -pattern_type glob -i '*.jpg' -vcodec libx264 -shortest '../../{OUTPUT_FILEPATH}'
%cd ../..